In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

years = np.arange(2008, 2016)

In [2]:
columns = ['Denominazione', 'Dizione_Provincia', 'Dizione_Regione', 'Dizione_zona']

final = pd.read_csv('Dati misc//Anagrafe_comuni.csv', sep = ';', encoding = 'latin_1', low_memory = False).set_index('Id_Ente')

final['Data_Istituzione'] = pd.to_datetime(final['Data_Istituzione'], format='%Y-%m-%d')
final = final[final['Data_Istituzione'].dt.year < 2008]

final = final[final['Data_Cessazione'].isna()]

final = final[final['Codice_Tipologia_DLGS_118_2011'] == 'ELCOMU'][columns]

final = final.rename(columns={'Denominazione': 'Comune', 'Dizione_Provincia': 'Provincia', 'Dizione_Regione': 'Regione', 'Dizione_zona': 'Zona'})
final.index.names = ['BDAP']

final

,Comune,Provincia,Regione,Zona
BDAP,,,,
111142930451558302,COMUNE DI VERMIGLIO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111142930456078402,COMUNE DI NARBOLIA,ORISTANO,SARDEGNA,ISOLE
111142930549694901,COMUNE DI RESANA,TREVISO,VENETO,NORD-EST
111242928159168602,COMUNE DI ALDINO,BOLZANO/BOZEN,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST
111342928035516202,COMUNE DI AUGUSTA,SIRACUSA,SICILIA,ISOLE
...,...,...,...,...
999842930521939101,COMUNE DI DOSOLO,MANTOVA,LOMBARDIA,NORD-OVEST
999842930529623202,COMUNE DI CESSOLE,ASTI,PIEMONTE,NORD-OVEST
999942930461543701,COMUNE DI BIENO,TRENTO,TRENTINO-ALTO ADIGE/SÜDTIROL,NORD-EST


In [3]:
final.to_pickle('comuni.pkl')

In [4]:
years = np.arange(2008, 2016)

# Zona

In [5]:
zona = pd.DataFrame(index = final.index)
for elem in final['Zona'].unique():
	zona[elem] = 0
for i, row in final.iterrows():
	zona.loc[i, row['Zona']] = 1

zona.to_pickle('zona.pkl')

# Popolazione

In [6]:
import os

def find_files(folder_path, search_string):
    matching_files = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if search_string in file:
                matching_files.append(os.path.join(root, file))

    return matching_files

def mean_age(data):
	 return np.mean(np.multiply(np.arange(0,101), data).sum())

In [7]:
comuni = pd.read_csv('Dati misc//Anagrafe_comuni.csv', sep = ';', encoding = 'latin_1', low_memory = False).set_index('Id_Ente')

In [8]:
popolazione = pd.DataFrame(index = final.index, columns = [f'pop_{year}' for year in years])

folder_path = 'popolazione//'

for idx in final.index:
	codice = str(comuni.loc[idx, 'Codice_ISTAT_Comune']).zfill(6)
	if codice == '099031':
		codice = '041060'
	if codice == '099030':
		codice = '041033'
	names = find_files(folder_path, codice)
	temp = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=106, nrows=1).values[0][7:15]
	popolazione.loc[idx] = temp
	
popolazione = popolazione.apply(pd.to_numeric, errors='coerce')

In [9]:
popolazione.to_pickle('pop.pkl')

In [10]:
eta = pd.DataFrame(index = final.index, columns = [f'eta_{year}' for year in years])

folder_path = 'popolazione//'

for idx in final.index:
	codice = str(comuni.loc[idx, 'Codice_ISTAT_Comune']).zfill(6)
	if codice == '099031':
		codice = '041060'
	if codice == '099030':
		codice = '041033'
	names = find_files(folder_path, codice)
	temp = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=4, nrows=102).values[1:,7:15].astype(int)
	temp2 = pd.read_csv(names[0], sep = ';', encoding = 'latin_1', low_memory = False, skiprows=106, nrows=1).values[0][7:15].astype(int)
	eta.loc[idx] = (temp* np.arange(temp.shape[0]).reshape(-1, 1)).sum(axis = 0)/temp2

eta = eta.apply(pd.to_numeric, errors='coerce')

In [11]:
eta.to_pickle('eta.pkl')

# Balance sheet items

In [ ]:
names = ['Attività e Passività', 'Costi e Ricavi', 'Entrate', 'Spese']

l1 = []
map1 = []
l2 = []
map2 = []

for name in names:
	filename = f'Dati 2008-2015//{name}.csv'
	bigdata = pd.read_csv(filename, sep = ';', encoding = 'latin_1', low_memory = False)
	
	if name == 'Costi e Ricavi':
		target = 'Valore Economico'
		slice1 = 'Descrizione CE I livello'
		slice2 = 'Descrizione CE II livello'
		
	elif name == 'Attività e Passività':
		target = 'Consistenza Finale Patrimonio'
		slice1 = 'Descrizione Voce I livello'
		slice2 = 'Descrizione Voce II livello'
		
	elif name == 'Entrate':
		target = 'Impegno totale'
		slice1 = 'Descrizione Titolo'
		slice2 = 'Descrizione Risorsa'
		
	else:
		target = 'Impegno totale'
		slice1 = 'Descrizione Titolo Spese'
		slice2 = 'Descrizione Intervento'
	
	print(f'############# DATASET:{name} #############')
	if name != 'Entrate':
		bigdata[target] = bigdata[target].str.replace(',', '.').astype(float)
	
	if name == 'Attività e Passività':
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Passivo') & (bigdata['Consistenza Finale Patrimonio'] < 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI ATTIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Attivo') & (bigdata['Consistenza Finale Patrimonio'] < 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI PASSIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Passivo') & (bigdata['Consistenza Finale Patrimonio'] >= 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI PASSIVO'
		bigdata.loc[(bigdata['Descrizione Voce I livello'] == 'RATEI E RISCONTI') & (bigdata['Tipologia Voce'] == 'Attivo') & (bigdata['Consistenza Finale Patrimonio'] >= 0), 'Descrizione Voce I livello'] = 'RATEI E RISCONTI ATTIVO'
		bigdata['Consistenza Finale Patrimonio'] = bigdata['Consistenza Finale Patrimonio'].abs()

	if name == 'Entrate':
		bigdata['Riscossione in CC'] = bigdata['Riscossione in CC'].str.replace(',', '.').astype(float)
		bigdata['Riscossione in CR'] = bigdata['Riscossione in CR'].str.replace(',', '.').astype(float)
		bigdata['Impegno totale'] = bigdata['Riscossione in CC'] + bigdata['Riscossione in CR']

	if name == 'Entrate':
		unique_slice1 = sorted(bigdata[slice1].unique()[:-1])
		unique_slice2 = sorted(bigdata[slice2].unique())

	if name != 'Entrate':
		unique_slice1 = sorted(bigdata[slice1].unique())
		unique_slice2 = sorted(bigdata[slice2].unique())
		
	print(f'CAT I items:{len(unique_slice1)}')
	l1 += list(unique_slice2)
	map1 += [name for elem in unique_slice1]
	print(f'CAT II items:{len(unique_slice2)}')
	l2 += list(unique_slice2)
	map2 += [name for elem in unique_slice2]
	
	for year in years:

		temp = bigdata.loc[bigdata['Esercizio Finanziario'] == year].copy()
		
		temp_df_1 = temp.pivot_table(index='Codice BDAP Ente', columns = slice1, values = target, aggfunc='sum', fill_value=0)
		temp_df_2 = temp.pivot_table(index='Codice BDAP Ente', columns = slice2, values = target, aggfunc='sum', fill_value=0)
		
		temp_df_1 = temp_df_1.reindex(columns=unique_slice1, fill_value=0)
		temp_df_2 = temp_df_2.reindex(columns=unique_slice2, fill_value=0)
		
		temp_df_1 = temp_df_1.reindex(final.index, fill_value=0)
		temp_df_2 = temp_df_2.reindex(final.index, fill_value=0)
		
		print(f'{name} {year} CAT I\tShape: {temp_df_1.shape}\tNaN: {temp_df_1.isnull().sum().sum()} ')
		print(f'{name} {year} CAT II\tShape: {temp_df_2.shape}\tNaN: {temp_df_2.isnull().sum().sum()} ')
		temp_df_1.to_pickle(f'{name} {year} CAT I.pkl')
		temp_df_2.to_pickle(f'{name} {year} CAT II.pkl')
		
		del temp_df_1
		del temp_df_2
		gc.collect()
		
	del bigdata
	gc.collect()